<a href="https://colab.research.google.com/github/milichowdhury/Machine_Chatbot/blob/main/DEPLOY_with_Collab(ngrok).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download necessary packages

In [1]:
pip install -r https://raw.githubusercontent.com/milichowdhury/Machine_Chatbot/main/requirements.txt > /dev/null 2>&1

#Import libraries

In [67]:
import os
import shutil
import getpass
import threading
import subprocess
from pyngrok import ngrok
import time
import signal

In [ ]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [ ]:
#CHECK if already cloned or not

# Path to the directory
directory_path = '/content/Machine_Chatbot'

# Check if the directory exists
if os.path.exists(directory_path):
    # Deleting the Machine_Chatbot directory
    shutil.rmtree(directory_path, ignore_errors=True)
    print(f"The directory '{directory_path}' has been deleted.")
else:
    print(f"The directory '{directory_path}' does not exist.")

In [ ]:
#only required if already existed and later got deleted
%cd /content/

In [77]:
!git clone https://github.com/milichowdhury/Machine_Chatbot.git > /dev/null 2>&1

In [ ]:
#check
%cd /content/Machine_Chatbot

In [ ]:
# Ask for ngrok token
ngrok_token = getpass.getpass('Enter your ngrok token: ')

# Authenticate ngrok
!ngrok authtoken {ngrok_token}

#Access it through tunnel where there is written ngrock-free.app and there click vist site

In [ ]:
def run_chainlit_app():
    # Start the Chainlit app
    subprocess.run(["chainlit", "run", "/content/Machine_Chatbot/src/data-analysis-llm-agent/app.py", "--port", "8002"])

# Run the Chainlit app in a separate thread
app_thread = threading.Thread(target=run_chainlit_app)
app_thread.start()

# Give some time for the app to start
time.sleep(5)

# Create a tunnel
public_url = ngrok.connect(8002)

print(f"App is accessible at {public_url}")

# Close the chainlit browser and run this to stop the process

In [ ]:
# Function to stop Chainlit and Ngrok
def stop_chainlit_and_ngrok():
    # Stop Chainlit by finding its process ID (PID)
    chainlit_port = 8002  # The port you started Chainlit on
    # Find the PID of the process running on port 8002 (Chainlit)
    stream = os.popen(f"lsof -i :{chainlit_port} -t")
    pid = stream.read().strip()

    if pid:
        print(f"Stopping Chainlit (PID: {pid})...")
        os.kill(int(pid), signal.SIGTERM)  # Gracefully terminate the process
        print("Chainlit stopped.")
    else:
        print(f"No process found on port {chainlit_port}.")

    # Stop Ngrok
    print("Disconnecting Ngrok...")
    ngrok.kill()  # Completely shut down Ngrok
    print("Ngrok disconnected and shut down.")

# Call the function to stop both Chainlit and Ngrok
stop_chainlit_and_ngrok()
